In [1]:
from keybert import KeyBERT
import whisper_timestamped as whisper
from nltk import download
from nltk.corpus import stopwords

download("stopwords")
YT_LINK = "/content/vid test.weba"

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Hardcoded text

In [7]:
with open("content/oleg.txt", "r", encoding="utf-8") as f:
    txt = f.read()

# NLP

In [8]:
from transformers import BertModel, BertTokenizerFast
from datasets import Dataset
import torch
import torch.nn.functional as F
import pandas as pd

In [55]:
model_checkpoint = 'bert-base-multilingual-cased'

tokenizer = BertTokenizerFast.from_pretrained(model_checkpoint)
model = BertModel.from_pretrained(model_checkpoint)
model = model.eval()

def get_embeddings(text):

  inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
  with torch.no_grad():
    outputs = model(**inputs)

  embeddings = outputs.pooler_output
  normalized_embeddings = F.normalize(embeddings, p=2)

  return normalized_embeddings

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

C:\Users\user\AppData\Local\pypoetry\Cache\virtualenvs\nlp-experements-i8WhhMzd-py3.10\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--bert-base-multilingual-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

In [56]:
kw_model = KeyBERT()

tokens = txt.split(" ")
indexed_keywords = {}
df = pd.DataFrame(columns=["index", "keywords"])
for i in range(50, len(tokens), 50):
  frag = tokens[i-50: i]
  frag = " ".join(frag)

  doc_embeddings, word_embeddings = kw_model.extract_embeddings(frag)
  keywords = kw_model.extract_keywords(frag,
                                     doc_embeddings=doc_embeddings,
                                     word_embeddings=word_embeddings)
  indexed_keywords[i] = frag

In [57]:
values_dict = {"index":[], "frags":[]}
for i, keywords in indexed_keywords.items():
  words = " ".join([k[0] for k in keywords])

  values_dict["index"].append(i)
  values_dict["frags"].append(keywords)

In [58]:
df = pd.DataFrame(values_dict)
df.head(5)

,index,frags
0,50,"Я очень рад представлять Олега Шатова, который..."
1,100,"часть произнесу, что очень рад поприветствоват..."
2,150,"вас покидаю.\nДа, я помучаю тебя вопросами, но..."
3,200,"сообщество, что там делаете интересного? Да, с..."
4,250,сотрудничества университета Итмо и компании На...


In [59]:
words_dataset = Dataset.from_pandas(df)
words_dataset = words_dataset.map(
    lambda x: {'text_embeddings': get_embeddings(x["frags"])[0]}
)

Map:   0%|          | 0/76 [00:00<?, ? examples/s]

In [60]:
words_dataset.to_pandas().head()

,index,frags,text_embeddings
0,50,"Я очень рад представлять Олега Шатова, который...","[0.048393887, -0.019466672, 0.019621193, -0.02..."
1,100,"часть произнесу, что очень рад поприветствоват...","[0.020003136, -0.015551811, 0.0002919152, -0.0..."
2,150,"вас покидаю.\nДа, я помучаю тебя вопросами, но...","[0.036019783, -0.01801094, 0.03005353, -0.0385..."
3,200,"сообщество, что там делаете интересного? Да, с...","[0.02511374, -0.019059332, 0.01801791, -0.0291..."
4,250,сотрудничества университета Итмо и компании На...,"[0.02118397, -0.0044696275, 0.012324263, -0.02..."


In [61]:
words_dataset = words_dataset.add_faiss_index(column='text_embeddings')

  0%|          | 0/1 [00:00<?, ?it/s]

In [83]:
test_frag_0 = words_dataset.to_pandas().head()['text_embeddings'][4]
words_dataset.to_pandas().head()

,index,frags,text_embeddings
0,50,"Я очень рад представлять Олега Шатова, который...","[0.048393887, -0.019466672, 0.019621193, -0.02..."
1,100,"часть произнесу, что очень рад поприветствоват...","[0.020003136, -0.015551811, 0.0002919152, -0.0..."
2,150,"вас покидаю.\nДа, я помучаю тебя вопросами, но...","[0.036019783, -0.01801094, 0.03005353, -0.0385..."
3,200,"сообщество, что там делаете интересного? Да, с...","[0.02511374, -0.019059332, 0.01801791, -0.0291..."
4,250,сотрудничества университета Итмо и компании На...,"[0.02118397, -0.0044696275, 0.012324263, -0.02..."


In [84]:
scores, samples = words_dataset.get_nearest_examples('text_embeddings',test_frag_0, k=15)


In [85]:
samples['index']

[250,
 2550,
 850,
 1300,
 2750,
 2650,
 450,
 200,
 2050,
 1500,
 3400,
 1100,
 1350,
 2400,
 150]